# Submitted by Akshata Mohan , Anurag Chilukuri, Matt Staton, Valerie Roth, Daan Alexander, Daniel Lindquist

# Part A 

We ran the numerical data with Multinomial Bayes and Logistic regression and obtained accuracy of 68% and 69% respectively. 

### Multinomial Bayes and logistic regression 

In [1244]:
from pandas import Series, DataFrame
import pandas as pd
import nltk
from nltk import word_tokenize
from collections import defaultdict
from collections import Counter
import operator
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from scipy.sparse import hstack
import scipy.sparse
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
%pylab inline

yelp_df = pd.read_csv("Yelp Data Restaurant Reviews Ratings.csv")

yelp_df.loc[yelp_df['stars'].isin([1,2,3])==True,'stars'] = 0

yelp_df.loc[yelp_df['stars'].isin([4,5])==True,'stars'] = 1


yelp_df_without_text = yelp_df.drop("Review", axis = 1, inplace = False)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [1254]:
train_yelp, test_yelp = train_test_split(yelp_df_without_text, test_size = 0.2)
logistic_model = linear_model.LogisticRegression(fit_intercept = True)
logistic_model = linear_model.LogisticRegression(fit_intercept = True)
test_y = test_yelp[["stars"]]
train_y = train_yelp[["stars"]]
train_yelp = train_yelp.drop("stars", axis = 1, inplace = False)
test_yelp = test_yelp.drop("stars", axis = 1,inplace = False)



In [1255]:

 
logistic_model.fit(train_yelp,train_y)
logistic_model.coef_

 
logistic_model.score(test_yelp,test_y)

C:\Users\anura\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.69074999999999998

In [1256]:
model_nb = MultinomialNB()
model_fit = model_nb.fit(train_yelp,train_y)

model_fit.score(test_yelp,test_y)

C:\Users\anura\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6825

# Part B


We ran a Multinomial Bayes model on only the reviews after removing all stopwords. We obtained an accuracy of 71.725% on unseen test set.

In [12]:
from pandas import Series, DataFrame
import pandas as pd
import nltk
from nltk import word_tokenize
from collections import defaultdict
from collections import Counter
import operator
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from scipy.sparse import hstack
import scipy.sparse
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
%pylab inline

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [13]:
yelp_df = pd.read_csv("Yelp Data Restaurant Reviews Ratings.csv")

In [14]:
yelp_df_without_text = yelp_df.drop("Review", axis = 1, inplace = False)
train_yelp, test_yelp = train_test_split(yelp_df_without_text, test_size = 0.2)

In [15]:
train_yelp = train_yelp.drop("stars", axis = 1, inplace = False)
test_yelp = test_yelp.drop("stars", axis = 1,inplace = False)

In [16]:
yelp_df.loc[yelp_df['stars'].isin([1,2,3])==True,'stars'] = 0

yelp_df.loc[yelp_df['stars'].isin([4,5])==True,'stars'] = 1 


In [17]:
for_yelp_only_text_train, for_yelp_only_text_test = train_test_split(yelp_df, test_size = 0.2)

In [18]:
yelp_only_text_train = for_yelp_only_text_train["Review"]
yelp_only_text_test = for_yelp_only_text_test["Review"]

In [19]:
stop_vectorizer = TfidfVectorizer(min_df=0, smooth_idf=True, strip_accents='unicode',\
                norm='l2', stop_words='english')

In [20]:
convert_df = np.array([''.join(el) for el in yelp_only_text_train])
df_text_train = stop_vectorizer.fit_transform(convert_df)

In [21]:
convert_df = np.array([''.join(el) for el in yelp_only_text_test])
df_text_test = stop_vectorizer.transform(convert_df)

In [22]:
labels_train = for_yelp_only_text_train["stars"]
labels_test = for_yelp_only_text_test["stars"]

In [23]:
model_nb = MultinomialNB()
model_fit = model_nb.fit(df_text_train, labels_train)

In [24]:
predictions = model_fit.predict(df_text_test)


In [25]:
predictions = pd.Series(predictions)
cm = confusion_matrix(predictions, labels_test)

In [26]:
cm

array([[ 138,    2],
       [1175, 2685]])

In [27]:
round((predictions == labels_test).mean() * 100,3)

70.575

# Part C 


We created a sparse matrix out of our numerical data and then used hstack() function in the scipy.sparse library to combine the two data matrices horizontally. ( combining the columns which are the attributes in numerical data matrix and the columns from the reviews which are the tf idf scores for each word in each document ). Then, we ran a Multinomial Bayes model to get an accuracy of 69.9% on unseen test set.

In [28]:
sparse_train_no_text = scipy.sparse.csr_matrix(train_yelp)

In [29]:
sparse_test_no_text = scipy.sparse.csr_matrix(test_yelp)

In [30]:
combined_train = scipy.sparse.hstack([df_text_train, sparse_train_no_text])

In [31]:
combined_test = scipy.sparse.hstack([df_text_test, sparse_test_no_text])

In [32]:
model_fit = model_nb.fit(combined_train, labels_train)

In [33]:
predictions_combined = model_fit.predict(combined_test)

In [34]:
cm_combined = confusion_matrix(predictions_combined, labels_test)

In [35]:
cm_combined

array([[ 113,    1],
       [1200, 2686]])

In [1223]:
round((predictions_combined == labels_test).mean() * 100,3)

69.975

### Comparision between the hybrid model and "only numeric" and "only reviews" models -

The best model accuracy was obtained with "only reviews" followed by the hybrid and then the numeric model. Although the percentage points do not vary too much, and they all have similar accuracy scores, we can take a guess that the "only numeric" model performs the "worst" because quantifying user preferences into numerical scores lead to loss of information. Whereas in a review, since the preferences are subjective in the sense the user types out what he/she feels, we could say that this would give a better indication of restaurant rating. The hybrid model is probably not performing exceptionally well because we could possibly be "overfitting" our model. The number of dimensions has increased considerably in our model and therefore, the accuracy on our test set is not as great because of the curse of dimentionality. ( Models that overfit will have "memorized" the data and therefore it will perform rather poorly on unseen test records )

#### Part D

We passed the first 1000 reviews in the text file through sentistrength and got the results. We arr adding the positve orientation and negative orientation to get the overall sentiment in a review. If it is greater than equal to 0, we are classifying them as high(1), else low(0)

In [1210]:

results=pd.read_table('C:\MSBA\Text Analytics\Assignment 2\Yelp 1000 reviews+results.txt')
results.columns = ['Review','Review1','Positive Orientation','Negative Orientation','breakup']
results['score']=results['Positive Orientation'] + results['Negative Orientation']
results['rating']=np.where(results['score']>=0,1,0)
results

,Review,Review1,Positive Orientation,Negative Orientation,breakup,score,rating
0,This is just a basic (albeit mini) chain greas...,This is just a basic (albeit mini) chain greas...,2,-2,This[0] is[0] just[0] a[0] basic[0] albeit[0]...,0,1
1,Whenever I offer to take my mom out to lunch s...,Whenever I offer to take my mom out to lunch s...,4,-4,Whenever[0] I[0] offer[0] to[0] take[0] my[0]...,0,1
2,If I say it wasn t as bad as I was expecting i...,If I say it wasn t as bad as I was expecting i...,3,-3,If[0] I[0] say[0] it[0] wasn[0] t[0] as[0] ba...,0,1
3,I ve always said if the guacamole chips and s...,I ve always said if the guacamole chips and s...,3,-3,I[0] ve[0] always[0] said[0] if[0] the[0] gua...,0,1
4,Had the signature Black Chile entree. It was ...,Had the signature Black Chile entree.<br> It ...,3,-1,Had[0] the[0] signature[0] Black[0] Chile[0] ...,2,1
5,After hitting up the bank to sign some paper w...,After hitting up the bank to sign some paper w...,4,-4,After[0] hitting[0] up[0] the[0] bank[0] to[0...,0,1
6,Great happy hour deals here! I loved the Cotij...,Great happy hour deals here!<br> I loved the C...,4,-5,Great[2] happy[1] hour[0] deals[0] here[0] [[...,-1,0
7,Fine. Just fine. C /B- average-- all around. ...,Fine.<br> Just fine.<br> C /B- average- all ar...,4,-3,"Fine[2] [[Sentence=-1,3=word max, 1-5]] Just[...",1,1
8,beautiful atmosphere...good prices (for the bi...,beautiful atmosphere.<br>good prices (for the ...,3,-1,"beautiful[2] atmosphere[0] [[Sentence=-1,3=wo...",2,1
9,Went there tonight. For a saturday night it w...,Went there tonight.<br> For a saturday night ...,2,-2,"Went[0] there[0] tonight[0] [[Sentence=-1,1=w...",0,1


In [1211]:
cm_d=confusion_matrix(results['rating'],yelp_df['new_rating'][:998])
cm_d

array([[ 32,  84],
       [255, 627]])

In [1212]:
((cm_d[0][0]+cm_d[1][1])*1.0)/(cm_d[0][1]+cm_d[1][0]+cm_d[0][0]+cm_d[1][1])

0.66032064128256518

We got an accuracy of 66.03% using SentiStrength. The accuracy is less than what we have got from Parts B and C. It is expected since this is a unsupervised learning model

## Part E

In [941]:
from nltk.corpus import stopwords
from nltk import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+')
yelp_df = pd.read_csv('C:\MSBA\Text Analytics\Assignment 2\Yelp Data Restaurant Reviews Ratings.csv') #Read the file
sample = yelp_df['Review'][:1000] #Take the first few samples
sample_decoded = [x.decode('utf-8') for x in sample] #decode the file
sample_decoded[:1]

[u'This location is out of business. I drove by it on my way to Costco and it just has a giant for lease sign.']

In [944]:
#Create tokens
sample_tokenized = []
for i in range(len(sample_decoded)):
    sample_tokenized.append(tokenizer.tokenize(sample_decoded[i]))
sample_tokenized[:1]

[[u'This',
  u'location',
  u'is',
  u'out',
  u'of',
  u'business',
  u'I',
  u'drove',
  u'by',
  u'it',
  u'on',
  u'my',
  u'way',
  u'to',
  u'Costco',
  u'and',
  u'it',
  u'just',
  u'has',
  u'a',
  u'giant',
  u'for',
  u'lease',
  u'sign']]

In [945]:
#Converts words to lowercase
sample_words_lower = []
lower_word_list=[]
for listofwords in sample_tokenized:
    for word in listofwords:
        lower_word_list.append(word.lower())
    sample_words_lower.append(lower_word_list)
    lower_word_list = []
sample_words_lower[:1]

[[u'this',
  u'location',
  u'is',
  u'out',
  u'of',
  u'business',
  u'i',
  u'drove',
  u'by',
  u'it',
  u'on',
  u'my',
  u'way',
  u'to',
  u'costco',
  u'and',
  u'it',
  u'just',
  u'has',
  u'a',
  u'giant',
  u'for',
  u'lease',
  u'sign']]

In [896]:
#Remove stopwords
sample_filtered = []  
no_stop =[]
l = []
for listofwords in sample_words_lower:
        listofwords = [word for word in listofwords if word not in stopwords.words('english')]
        sample_filtered.append(listofwords)

In [946]:
sample_filtered[:1]

[[u'location',
  u'business',
  u'drove',
  u'way',
  u'costco',
  u'giant',
  u'lease',
  u'sign']]

In [947]:
#Convert the output to a series
yrs = pd.Series(sample_filtered)
yrs[:5]

0    [location, business, drove, way, costco, giant...
1    [closed, jb, location, poor, building, layout,...
2    [basic, albeit, mini, chain, greasy, spoon, de...
3    [whenever, offer, take, mom, lunch, somewhere,...
4    [say, bad, expecting, good, 2, 5, stars, eaten...
dtype: object

In [898]:
#create parts of speech tags
yrs_tag=[]
for i in yrs:
    yrs_tag.append(nltk.pos_tag(i))

In [950]:
yrs_tag[:1]

[[(u'location', 'NN'),
  (u'business', 'NN'),
  (u'drove', 'VBD'),
  (u'way', 'NN'),
  (u'costco', 'NN'),
  (u'giant', 'JJ'),
  (u'lease', 'NN'),
  (u'sign', 'NN')]]

In [899]:
#extract words based on POS rules defined in the research paper
extracted_words=[]
z=[]
for i in range(len(yrs_tag)):
    for (w1,t1), (w2,t2),(w3,t3) in nltk.trigrams(yrs_tag[i]):
        if ((t1=='JJ' and t2==('NN' or 'NNS')) or (t1==('RB' or 'RBR' or 'RBS') and t2=='JJ' and t3!=('NN' or 'NNS')) or (t1=='JJ' and t2=='JJ' and t3!=('NN' or 'NNS')) or (t1==('NN' or 'NNS') and t2=='JJ' and t3!=('NN' or 'NNS')) or (t1==('RB' or 'RBR' or'RBS') and t2==('VB' or 'VBD' or 'VBN' or 'VBG'))) :
            z.append(w1+' '+w2)
    extracted_words.append(z)
    z=[]

In [953]:
extracted_words[:3]

[[u'giant lease'],
 [u'closed jb', u'poor building', u'poor service'],
 [u'basic albeit',
  u'greasy spoon',
  u'decent burger',
  u'good service',
  u'relatively low',
  u'nearby costco']]

Create a cooccurence matrix - com which contains words within the bigrams + words adjacent to it - +-1
Here, the procedure deviates slightly from the research paper - We pick up the words within the bigram as well to calculate the sentiment- It is done to safegaurd context - If we have to evaluate the sentence - 'poor building good food' for the bigrams - poor building and good food. It makes sense to evaluate poor building using poor and good food using good.

In [900]:
#Create a cooccurence matrix - com which contains words within the bigrams + words adjacent to it - +-1
com = defaultdict(lambda : defaultdict(int))
for i in range(len(extracted_words[:1000])):
    for m in range(len(extracted_words[i])):
            a = []
            for b in nltk.bigrams(yrs[i]):
                a.append((b[0]+" "+b[1]))
            index_word = a.index(extracted_words[i][m])
            for j in range(max(0,index_word-1),min(len(yrs[i]),index_word+3)):
                w1, w2 = [extracted_words[i][m], yrs[i][j]] 
                com[w1][w2] += 1

In [956]:
com

defaultdict(<function __main__.<lambda>>,
            {u'piccata bad': defaultdict(int,
                         {u'also': 1,
                          u'bad': 1,
                          u'chicken': 1,
                          u'piccata': 1}),
             5: defaultdict(int, {}),
             u'hot crispy': defaultdict(int,
                         {u'better': 1,
                          u'crispy': 1,
                          u'hot': 1,
                          u'outside': 1}),
             u'upscale tomaso': defaultdict(int,
                         {u'restaurant': 1,
                          u'right': 1,
                          u'tomaso': 1,
                          u'upscale': 1}),
             u'close resemblance': defaultdict(int,
                         {u'close': 1,
                          u'el': 1,
                          u'place': 1,
                          u'resemblance': 1}),
             u'bit sweet': defaultdict(int,
                         {u'bit': 2,
 

In [901]:
#Use sentiment file from sentistrength to get the scores and lable words as positive or negative
sentiment = pd.read_csv('C:\MSBA\Text Analytics\Assignment 2\Sentiment.csv')
sentiment.columns=['words','score']

In [957]:
#Convert com to a data frame
df_com=pd.DataFrame(com)
df_com[:5]

,5,20th street,21st century,4th time,50th birthday,_ carlos,aa phone,abayas young,able answer,able call,...,yummy little,yummy meal,yummy medium,yummy place,yummy salsa,yummy service,zesty quesadilla,zillionth time,zombie hubby,zombie hurricane
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [961]:
#Get the score of words by joining with the sentiment dataframe
df_scored=df_com.merge(sentiment,how = 'left',left_index=True,right_on=np.array(sentiment['words']))


In [905]:
#Converts scores to sentiments
def senti_class(x):
    if x>0:
        return 'positive'
    elif x<0:
        return 'negative'
    elif x==np.nan:
        return 'neutral'

In [963]:
#Creates sentiment for each word
df_scored['sentiment']=df_scored['score'].map(senti_class)

In [964]:
#Calculates the probability of positive and negative
prob_positive=(len(df_scored[df_scored['sentiment']=='positive'])*1.0)/len(extracted_words)
prob_negative=(len(df_scored[df_scored['sentiment']=='negative'])*1.0)/len(extracted_words)
print prob_positive,prob_negative

0.084 0.154


In [966]:
#Counts the number of bigrams - divides by 4 because each bigram will have 4 mentions
count_bigrams=pd.DataFrame(df_scored.sum()/4)

In [1050]:
#names the columns and converts bigrams to a column
count_bigrams.columns=['counts']
count_bigrams['bigrams']=count_bigrams.index

In [1051]:
#Calculates the presence of a bigram along with a positive word
with_positives=pd.DataFrame((df_scored[df_scored['sentiment']=='positive'].drop(['key_0','words','score','sentiment'],1).sum()))
with_positives.columns=['counts_positive']
with_positives['bigrams']=with_positives.index

#Calculates the presence of a bigram with a negative word
with_negatives=pd.DataFrame((df_scored[df_scored['sentiment']=='negative'].drop(['key_0','words','score','sentiment'],1).sum()))
with_negatives.columns=['counts_negative']
with_negatives['bigrams']=with_negatives.index



positive_so represents positive semantic orientation - It is calculated as follows - log2((bigrams mentioned together with a positive word)/(total mentions of the bigrams * probability of a positive word) 

Just the counts are taken while calculating the positive_so as the sample size is the same, ratio of probabilities will be equal to the ration of counts

negative_so is also calculated similarly and finally semantic orientation of the phase is calculated by subtracting the negative_so from positive_so


In [1208]:
master=count_bigrams.merge(with_positives,how='left',left_index=True,right_index=True).merge(with_negatives,how='left',left_index=True,right_index='True')[['counts','counts_positive','counts_negative']]
master['positive_so']=np.log2((((master['counts_positive']*1.0)+0.01)/(master['counts']*prob_positive)))
master['negative_so']=np.log2((((master['counts_negative']*1.0)+0.01)/(master['counts']*prob_negative)))
master['semantic_orientation']=master['positive_so']-master['negative_so']
master[:5]

,counts,counts_positive,counts_negative,positive_so,negative_so,semantic_orientation
5,0.00,0.0,0.0,inf,inf,NaN
20th street,1.00,0.0,0.0,-3.070389,-3.944858,0.874469
21st century,1.00,0.0,0.0,-3.070389,-3.944858,0.874469
4th time,0.75,0.0,0.0,-2.655352,-3.529821,0.874469
50th birthday,1.00,0.0,0.0,-3.070389,-3.944858,0.874469


In [968]:
df_computed=pd.DataFrame(pd.Series(extracted_words))
df_computed.columns=['bigrams']
#A function that sums up the semantic orientations and returns the net semantic orientation of a review
def so_addition(x):
    so_total = 0
    so_total += master.ix[x,'semantic_orientation']  
    return so_total.sum()


In [969]:
df_computed['review_semantic_score']=df_computed['bigrams'].map(so_addition)

In [970]:
#predicts rating for reviews - 1(high) if semantic orientation is greater than 0, else 0(low)
df_computed['rating']=np.where(df_computed['review_semantic_score']>0,1,0)

In [971]:
yelp_df['new_rating'] = np.where(yelp_df['stars']<=3,0,1)

In [972]:
cm_e = confusion_matrix(df_computed['rating'],yelp_df['new_rating'][:1000])
cm_e

array([[ 21,  31],
       [267, 681]])

In [1224]:
accuracy = (cm_e[0][0]+cm_e[1][1])*1.0/(cm_e[0][0]+cm_e[0][1]+cm_e[1][0]+cm_e[1][1])
accuracy

0.70199999999999996

The results are better than Part D. 

## Part F

Approach - The data is split into high rated and low rated. Then we use PMI to calculate the semantic orientation of single words and obtain top 10 words with high semantic orientation and bottom 10 words with low semantic orientation. This tells us which aspects contribute positively and what contriute negatively to a high rated restaurant. 

In [1031]:
df_yrs=pd.DataFrame(yrs)
df_yrs.columns=['review words']
df_yrs['ratings']=yelp_df['new_rating'][:1000]
df_yrs_high=df_yrs[df_yrs['ratings']==1]
df_yrs_high.index=range(0,len(df_yrs_high))
df_yrs_high[:5]

,review words,ratings
0,"[basic, albeit, mini, chain, greasy, spoon, de...",1
1,"[signature, black, chile, entree, fabulous]",1
2,"[great, happy, hour, deals, loved, cotija, gri...",1
3,"[beautiful, atmosphere, good, prices, biltmore...",1
4,"[went, tonight, saturday, night, full, table, ...",1


In [988]:
yrs_high_tag=[]
for i in df_yrs_high['review words']:
    yrs_high_tag.append(nltk.pos_tag(i))

In [998]:
extracted_words_high=[]
z=[]
for i in range(len(yrs_high_tag)):
    for (w1,t1), (w2,t2),(w3,t3) in nltk.trigrams(yrs_high_tag[i]):
        if ((t1=='JJ' and t2==('NN' or 'NNS')) or (t1==('RB' or 'RBR' or 'RBS') and t2=='JJ' and t3!=('NN' or 'NNS')) or (t1=='JJ' and t2=='JJ' and t3!=('NN' or 'NNS')) or (t1==('NN' or 'NNS') and t2=='JJ' and t3!=('NN' or 'NNS')) or (t1==('RB' or 'RBR' or'RBS') and t2==('VB' or 'VBD' or 'VBN' or 'VBG'))) :
            z.append(w2)
    extracted_words_high.append(z)
    z=[]

In [999]:
extracted_words_high[:1]

[[u'albeit', u'spoon', u'burger', u'service', u'low', u'costco']]

In [1022]:
#Create a cooccurence matrix - com which contains words within the words + words adjacent to it - +-1
com_high = defaultdict(lambda : defaultdict(int))
for i in range(len(extracted_words_high[:1000])):
    for m in range(len(extracted_words_high[i])):
            a=df_yrs_high['review words'][i]
            index_word = a.index(extracted_words_high[i][m])
            for j in range(max(0,index_word-1),min(len(yrs[i]),index_word+3)):
                w1, w2 = [extracted_words_high[i][m], yrs[i][j]] 
                com_high[w1][w2] += 1

In [1028]:
com_high

defaultdict(<function __main__.<lambda>>,
            {u'limited': defaultdict(int,
                         {u'appetizers': 1,
                          u'back': 1,
                          u'bbq': 1,
                          u'beef': 1,
                          u'crispy': 1,
                          u'fish': 1,
                          u'full': 1,
                          u'meal': 1,
                          u'never': 1,
                          u'review': 1,
                          u'two': 1,
                          u'went': 1}),
             3: defaultdict(int, {}),
             4: defaultdict(int, {}),
             u'forget': defaultdict(int,
                         {u'beliefs': 1,
                          u'complaining': 1,
                          u'experience': 1,
                          u'feel': 1,
                          u'prevent': 1,
                          u'returning': 1,
                          u'weird': 1,
                          u'would': 1}),


In [1030]:
df_com_high=pd.DataFrame(com_high)
df_com_high[:5]

,3,4,able,accent,acidity,act,addition,additional,admit,adventure,...,yin,yoga,yogurt,york,yorkie,young,yum,yummy,zombie,zoolbia
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1032]:
df_scored_high=df_com_high.merge(sentiment,how = 'left',left_index=True,right_on=np.array(sentiment['words']))

In [1033]:
df_scored_high['sentiment']=df_scored_high['score'].map(senti_class)

In [1034]:
#Calculates the probability of positive and negative
prob_positive_high=(len(df_scored_high[df_scored_high['sentiment']=='positive'])*1.0)/len(extracted_words_high)
prob_negative_high=(len(df_scored_high[df_scored_high['sentiment']=='negative'])*1.0)/len(extracted_words_high)
print prob_positive_high,prob_negative_high

0.102528089888 0.115168539326


In [1037]:
count_words_high=pd.DataFrame(df_scored_high.sum()/4)

count_words_high.columns=['counts']
count_words_high['words']=count_words_high.index

#Calculates the presence of a bigram along with a positive word
with_positives_high=pd.DataFrame((df_scored_high[df_scored_high['sentiment']=='positive'].drop(['key_0','words','score','sentiment'],1).sum()))
with_positives_high.columns=['counts_positive']
with_positives_high['words']=with_positives_high.index

#Calculates the presence of a bigram with a negative word
with_negatives_high=pd.DataFrame((df_scored_high[df_scored_high['sentiment']=='negative'].drop(['key_0','words','score','sentiment'],1).sum()))
with_negatives_high.columns=['counts_negative']
with_negatives_high['words']=with_negatives_high.index

In [1151]:
master_high=count_words_high.merge(with_positives_high,how='left',left_index=True,right_index=True).merge(with_negatives_high,how='left',left_index=True,right_index='True')[['counts','counts_positive','counts_negative']]
master_high['positive_so']=np.log2((((master_high['counts_positive']*1.0)+0.01)/(master_high['counts']*prob_positive_high)))
master_high['negative_so']=np.log2((((master_high['counts_negative']*1.0)+0.01)/(master_high['counts']*prob_negative_high)))
master_high['semantic_orientation']=master_high['positive_so']-master_high['negative_so']

In [1170]:
master_high.dropna() #drops na

,counts,counts_positive,counts_negative,positive_so,negative_so,semantic_orientation
able,3.00,0.0,0.0,-4.942910,-5.110637,0.167727
accent,1.00,0.0,0.0,-3.357947,-3.525675,0.167727
acidity,1.00,1.0,0.0,3.300264,-3.525675,6.825939
act,1.00,0.0,0.0,-3.357947,-3.525675,0.167727
addition,5.00,1.0,0.0,0.978336,-5.847603,6.825939
additional,1.00,0.0,0.0,-3.357947,-3.525675,0.167727
admit,1.00,0.0,0.0,-3.357947,-3.525675,0.167727
adventure,2.00,1.0,0.0,2.300264,-4.525675,6.825939
adventurous,2.00,0.0,0.0,-4.357947,-4.525675,0.167727
affable,1.00,0.0,0.0,-3.357947,-3.525675,0.167727


In [1213]:
#We create pos_tags using nltk and take out only nouns
pos_tag=pd.DataFrame(nltk.pos_tag(master_high.dropna().index.values))
pos_tag.columns=['words','pos_tag']
master_tagged=master_high.merge(pos_tag,how='left',left_index='True',right_on=np.array(pos_tag['words']))
master_filtered=master_tagged[master_tagged['pos_tag'].isin(['NN','NNS'])]
master_filtered.sort_values(by='semantic_orientation',ascending=False)['key_0'][:10]

1118    restaurant
821          lunch
846           meal
514         flavor
675           hula
1464          wait
1399      tropical
1493          wife
202        ceviche
1370          tiki
Name: key_0, dtype: object

The above words contribute positively to the highly rated restaurants

In [1214]:
master_filtered.sort_values(by='semantic_orientation',ascending=True)['key_0'][:10]

835          market
96        bartender
58          artwork
289        convince
728             job
1244            son
1145           rush
549           funky
1487      welcoming
280     consistency
Name: key_0, dtype: object

The above words contribute negatively to the low rated restaurant

In [ ]:
yrs_low_tag=[]
for i in df_yrs_low['review words']:
    yrs_low_tag.append(nltk.pos_tag(i))

In [1081]:
extracted_words_low=[]
z=[]
for i in range(len(yrs_low_tag)):
    for (w1,t1), (w2,t2),(w3,t3) in nltk.trigrams(yrs_low_tag[i]):
        if ((t1=='JJ' and t2==('NN' or 'NNS')) or (t1==('RB' or 'RBR' or 'RBS') and t2=='JJ' and t3!=('NN' or 'NNS')) or (t1=='JJ' and t2=='JJ' and t3!=('NN' or 'NNS')) or (t1==('RB' or 'RBR' or'RBS') and t2==('VB' or 'VBD' or 'VBN' or 'VBG'))) :
            z.append(w2)
        elif (t1==('NN' or 'NNS') and t2=='JJ' and t3!=('NN' or 'NNS')):
            z.append(w1)
    extracted_words_low.append(z)
    z=[]

extracted_words_low[:1]

[[u'lease']]

In [1082]:
#Create a cooccurence matrix - com which contains words within the words + words adjacent to it - +-1
com_low = defaultdict(lambda : defaultdict(int))
for i in range(len(extracted_words_low[:1000])):
    for m in range(len(extracted_words_low[i])):
            a=df_yrs_low['review words'][i]
            index_word = a.index(extracted_words_low[i][m])
            for j in range(max(0,index_word-1),min(len(yrs[i]),index_word+3)):
                w1, w2 = [extracted_words_low[i][m], yrs[i][j]] 
                com_low[w1][w2] += 1

In [1083]:
com_low

defaultdict(<function __main__.<lambda>>,
            {u'13th': defaultdict(int,
                         {u'evening': 1, u'joint': 1, u'sat': 1, u'slid': 1}),
             u'accomodate': defaultdict(int,
                         {u'asada': 1, u'carne': 1, u'tacos': 1, u'try': 1}),
             u'acknowledge': defaultdict(int,
                         {u'lesbian': 1,
                          u'lover': 1,
                          u'murdering': 1,
                          u'sleep': 1}),
             u'adaptive': defaultdict(int,
                         {u'given': 1,
                          u'immediately': 1,
                          u'menus': 1,
                          u'seated': 1}),
             u'addiction': defaultdict(int,
                         {u'could': 2, u'menu': 2, u'ordered': 2, u'try': 2}),
             u'addition': defaultdict(int,
                         {u'1': 1,
                          u'actually': 1,
                          u'complimented': 1,
          

In [1085]:
df_com_low=pd.DataFrame(com_low)
df_com_low[:5]

,13th,accomodate,acknowledge,adaptive,addiction,addition,additional,adequate,affected,afternoon,...,yeah,year,yelp,yogurt,yolk,york,young,yummy,zest,zombie
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1086]:
df_scored_low=df_com_low.merge(sentiment,how = 'left',left_index=True,right_on=np.array(sentiment['words']))

In [1087]:
df_scored_low['sentiment']=df_scored_low['score_y'].map(senti_class)

In [1088]:
#Calculates the probability of positive and negative
prob_positive_low=(len(df_scored_low[df_scored_low['sentiment']=='positive'])*1.0)/len(extracted_words_low)
prob_negative_low=(len(df_scored_low[df_scored_low['sentiment']=='negative'])*1.0)/len(extracted_words_low)
print prob_positive_low,prob_negative_low

0.184027777778 0.145833333333


In [1089]:
count_words_low=pd.DataFrame(df_scored_low.sum()/4)

In [1090]:
count_words_low.columns=['counts']
count_words_low['words']=count_words_low.index

In [1091]:
#Calculates the presence of a bigram along with a positive word
with_positives_low=pd.DataFrame((df_scored_low[df_scored_low['sentiment']=='positive'].drop(['key_0','words','score_y','sentiment'],1).sum()))
with_positives_low.columns=['counts_positive']
with_positives_low['words']=with_positives_low.index

#Calculates the presence of a bigram with a negative word
with_negatives_low=pd.DataFrame((df_scored_low[df_scored_low['sentiment']=='negative'].drop(['key_0','words','score_y','sentiment'],1).sum()))
with_negatives_low.columns=['counts_negative']
with_negatives_low['words']=with_negatives_low.index

In [1216]:
master_low=count_words_low.merge(with_positives_low,how='left',left_index=True,right_index=True).merge(with_negatives_low,how='left',left_index=True,right_index='True')[['counts','counts_positive','counts_negative']]
master_low['positive_so']=np.log2((((master_low['counts_positive']*1.0)+0.01)/(master_low['counts']*prob_positive_low)))
master_low['negative_so']=np.log2((((master_low['counts_negative']*1.0)+0.01)/(master_low['counts']*prob_negative_low)))
master_low['semantic_orientation']=master_low['positive_so']-master_low['negative_so']

In [1219]:
pos_tag=pd.DataFrame(nltk.pos_tag(master_low.index.values))
pos_tag.columns=['words','pos_tag']
pos_tag
master_tagged=master_low.merge(pos_tag,how='left',left_index='True',right_on=np.array(pos_tag['words']))
master_filtered=master_tagged[master_tagged['pos_tag'].isin(['NN','NNS'])]
master_filtered.sort_values(by='semantic_orientation',ascending=False)['key_0'][:10]

381      hour
217      dish
662    return
489      meal
136     chile
754     staff
611    potato
667      rice
869    waiter
394      idea
Name: key_0, dtype: object

The above words contribute positively to the low rated restaurants

In [1218]:
master_filtered.sort_values(by='semantic_orientation',ascending=True)['key_0'][:10]

493          meet
43       attitude
749         split
866    visitation
9       afternoon
395     immediate
254         enter
364         henna
112        cannot
707        server
Name: key_0, dtype: object

The above words contribute negatively to the low rated restaurants